In [55]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import scipy

In [56]:

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory= r"C:\Users\47573400\Downloads\dataset", target_size=(50,50))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=r"C:\Users\47573400\Downloads\dataset", target_size=(50,50))

Found 33762 images belonging to 2 classes.
Found 33762 images belonging to 2 classes.


In [57]:
#16 boxes de 50x50
testdata._get_batches_of_transformed_samples([1, ['1b1B1Qr1-7p-6r1-2P5-4Rk2-1K6-4B3-8']])
#tf.image.crop_to_bounding_box(testdata.classes[0]imag, 0, 0, 2, 2)
#tf.image.crop_to_bounding_box(testdata)

NameError: name 'scipy' is not defined

https://builtin.com/machine-learning/vgg16